In [1]:
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import time
import progressBar
from imageai.Detection import ObjectDetection

/Users/frankturner/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#import locations
image_directory = pd.read_csv("./image_directory.csv")
image_directory=image_directory[['fashion_image','designer','season','look','file_path']]
image_directory.tail()

,fashion_image,designer,season,look,file_path
15538,https://assets.vogue.com/photos/55c651d108298d...,Gucci,Fall 2015 Menswear,Look 32,./vogue_runway/Gucci/Fall 2015 Menswear/Gucci_...
15539,https://assets.vogue.com/photos/55c651d108298d...,Gucci,Fall 2015 Menswear,Look 33,./vogue_runway/Gucci/Fall 2015 Menswear/Gucci_...
15540,https://assets.vogue.com/photos/55c651d108298d...,Gucci,Fall 2015 Menswear,Look 34,./vogue_runway/Gucci/Fall 2015 Menswear/Gucci_...
15541,https://assets.vogue.com/photos/55c651d108298d...,Gucci,Fall 2015 Menswear,Look 35,./vogue_runway/Gucci/Fall 2015 Menswear/Gucci_...
15542,https://assets.vogue.com/photos/55c651d108298d...,Gucci,Fall 2015 Menswear,Look 36,./vogue_runway/Gucci/Fall 2015 Menswear/Gucci_...


In [3]:
#tensorflow will detect people, even if they are in the crowd
#since the model will be front and center, I can expect the image in 
#this directory that is the largest, will be the model

def find_the_model(detections):
    biggest_file_size=0
    biggest_image=''
    for i,person in enumerate(detections[0]):
        image_path=detections[1][i]
        if person['name']=='person':
            file_size=os.stat(image_path).st_size
            if file_size>biggest_file_size:
                biggest_file_size=file_size
                biggest_image = image_path
            else:os.remove(image_path)
        else:os.remove(image_path)
    return biggest_image


In [4]:
#Compresses the images and saves the compressed image to a new directory
compressed_directory = pd.DataFrame(columns=['compressed_image_filepath'])
tensor_flow_detected_model_directory = pd.DataFrame(columns=['detected_image_filepath'])
runtime_errors = []

In [5]:
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("./resnet50_coco_best_v2.0.1.h5")
detector.loadModel()

start = time.time()

for image_index in range(1297,image_directory.shape[0]):#start at 1297 because of debugging. 0-1296 was run below
#for image_index in range(1):#for running once, debugging
    filename = image_directory.file_path[image_index]
    designer = image_directory.designer[image_index]
    season= image_directory.season[image_index]
    look = image_directory.look[image_index]
    
    #compress the image and save it to a new folder
    image = cv2.imread(filename,cv2.IMREAD_COLOR)
    compressed_image=cv2.resize(image,None,fx=0.1,fy=0.1)
    compressed_image_filepath='./compressed_images/{}_{}_{}_compressed.jpg'.format(designer,season,look)
    cv2.imwrite(compressed_image_filepath, compressed_image)
    compressed_directory=compressed_directory.append(pd.DataFrame(data=[compressed_image_filepath],columns=['compressed_image_filepath'],index=[image_index]))
    #0.09733414649963379 for 1 iteration to get to here
    #~25 minutes 21 seconds for 15,000 images 
    
    
    #pick out the models using tensorflow
    try:
        detections = detector.detectObjectsFromImage(input_image=compressed_image_filepath,
                                                output_image_path="./close_up/new.jpg",
                                                extract_detected_objects=True)
    except (RuntimeError,SystemError,ValueError): 
        print(compressed_image_filepath+' caused a Runtime Error')
        runtime_errors.append(compressed_image_filepath)
        continue
        
    model_image=find_the_model(detections)
    new_path = "./tensor_flow_detected_models/{}_{}_{}_detected.jpg".format(designer,season,look)
    try: 
        os.rename(model_image,new_path)
    except FileNotFoundError:
        print(compressed_image_filepath+' caused a File Not Found Error')
        runtime_errors.append(compressed_image_filepath)
        continue
    tensor_flow_detected_model_directory=tensor_flow_detected_model_directory.append(pd.DataFrame(data=[new_path],columns=['detected_image_filepath'],index=[image_index]))
    #7.613460063934326 for 1 iteration to get to here
    #32.868998976018695 hours for 15,542 images 
    #12.986150979995728 seconds for uncompressed images,taking 56.06409959197045 hours
    print(str(image_index)+' of '+ str(image_directory.shape[0]))
end = time.time()
compressed_directory.to_csv('./compressed_images/directory.csv')
tensor_flow_detected_model_directory.to_csv('./tensor_flow_detected_models/directory.csv')
print(end - start)
 

1297 of 15543
./compressed_images/Versace_Fall 2015 Ready-to-Wear_Look 53_compressed.jpg caused a File Not Found Error
1299 of 15543
1300 of 15543
1301 of 15543
1302 of 15543
1303 of 15543
1304 of 15543
1305 of 15543
1306 of 15543
1307 of 15543
1308 of 15543
1309 of 15543
1310 of 15543
1311 of 15543
1312 of 15543
1313 of 15543
1314 of 15543
1315 of 15543
1316 of 15543
./compressed_images/Versace_Spring 2015 Menswear_Look 19_compressed.jpg caused a Runtime Error
1318 of 15543
1319 of 15543
1320 of 15543
1321 of 15543
1322 of 15543
1323 of 15543
1324 of 15543
1325 of 15543
1326 of 15543
1327 of 15543
1328 of 15543
1329 of 15543
1330 of 15543
1331 of 15543
1332 of 15543
1333 of 15543
1334 of 15543
1335 of 15543
1336 of 15543
1337 of 15543
1338 of 15543
1339 of 15543
1340 of 15543
1341 of 15543
1342 of 15543
./compressed_images/Versace_Spring 2015 Menswear_Look 45_compressed.jpg caused a Runtime Error
1344 of 15543
1345 of 15543
1346 of 15543
1347 of 15543
1348 of 15543
1349 of 15543
1350 

1755 of 15543
1756 of 15543
1757 of 15543
1758 of 15543
1759 of 15543
1760 of 15543
1761 of 15543
1762 of 15543
1763 of 15543
1764 of 15543
1765 of 15543
1766 of 15543
1767 of 15543
1768 of 15543
1769 of 15543
1770 of 15543
1771 of 15543
1772 of 15543
1773 of 15543
1774 of 15543
1775 of 15543
1776 of 15543
1777 of 15543
1778 of 15543
1779 of 15543
1780 of 15543
1781 of 15543
1782 of 15543
1783 of 15543
1784 of 15543
1785 of 15543
1786 of 15543
1787 of 15543
1788 of 15543
1789 of 15543
1790 of 15543
1791 of 15543
1792 of 15543
1793 of 15543
1794 of 15543
1795 of 15543
1796 of 15543
1797 of 15543
1798 of 15543
1799 of 15543
1800 of 15543
1801 of 15543
1802 of 15543
1803 of 15543
1804 of 15543
1805 of 15543
1806 of 15543
1807 of 15543
1808 of 15543
1809 of 15543
1810 of 15543
1811 of 15543
1812 of 15543
1813 of 15543
1814 of 15543
1815 of 15543
1816 of 15543
1817 of 15543
1818 of 15543
1819 of 15543
1820 of 15543
1821 of 15543
1822 of 15543
1823 of 15543
1824 of 15543
1825 of 15543
1826 o

2273 of 15543
2274 of 15543
2275 of 15543
2276 of 15543
2277 of 15543
2278 of 15543
2279 of 15543
2280 of 15543
2281 of 15543
./compressed_images/Koché_Pre-Fall 2018_Look 4_compressed.jpg caused a Runtime Error
2283 of 15543
2284 of 15543
2285 of 15543
2286 of 15543
2287 of 15543
2288 of 15543
2289 of 15543
2290 of 15543
2291 of 15543
2292 of 15543
2293 of 15543
2294 of 15543
2295 of 15543
2296 of 15543
2297 of 15543
./compressed_images/Koché_Pre-Fall 2018_Look 20_compressed.jpg caused a Runtime Error
2299 of 15543
2300 of 15543
2301 of 15543
2302 of 15543
2303 of 15543
2304 of 15543
2305 of 15543
2306 of 15543
2307 of 15543
2308 of 15543
2309 of 15543
2310 of 15543
2311 of 15543
2312 of 15543
./compressed_images/Koché_Pre-Fall 2018_Look 35_compressed.jpg caused a Runtime Error
2314 of 15543
2315 of 15543
2316 of 15543
2317 of 15543
2318 of 15543
2319 of 15543
2320 of 15543
2321 of 15543
./compressed_images/Koché_Pre-Fall 2018_Look 44_compressed.jpg caused a Runtime Error
2323 of 15543

./compressed_images/Saint Laurent_Spring 2018 Ready-to-Wear_Look 59_compressed.jpg caused a Runtime Error
2751 of 15543
2752 of 15543
2753 of 15543
./compressed_images/Saint Laurent_Spring 2018 Ready-to-Wear_Look 63_compressed.jpg caused a Runtime Error
2755 of 15543
2756 of 15543
2757 of 15543
2758 of 15543
2759 of 15543
2760 of 15543
2761 of 15543
2762 of 15543
2763 of 15543
2764 of 15543
./compressed_images/Saint Laurent_Spring 2018 Ready-to-Wear_Look 74_compressed.jpg caused a Runtime Error
2766 of 15543
2767 of 15543
2768 of 15543
2769 of 15543
2770 of 15543
2771 of 15543
2772 of 15543
2773 of 15543
2774 of 15543
2775 of 15543
2776 of 15543
2777 of 15543
2778 of 15543
./compressed_images/Saint Laurent_Spring 2018 Ready-to-Wear_Look 88_compressed.jpg caused a Runtime Error
2780 of 15543
2781 of 15543
2782 of 15543
2783 of 15543
2784 of 15543
2785 of 15543
2786 of 15543
2787 of 15543
2788 of 15543
2789 of 15543
2790 of 15543
2791 of 15543
2792 of 15543
2793 of 15543
2794 of 15543
27

./compressed_images/Saint Laurent_Spring 2016 Ready-to-Wear_Look 25_compressed.jpg caused a File Not Found Error
3124 of 15543
3125 of 15543
3126 of 15543
./compressed_images/Saint Laurent_Spring 2016 Ready-to-Wear_Look 29_compressed.jpg caused a File Not Found Error
./compressed_images/Saint Laurent_Spring 2016 Ready-to-Wear_Look 30_compressed.jpg caused a File Not Found Error
3129 of 15543
3130 of 15543
./compressed_images/Saint Laurent_Spring 2016 Ready-to-Wear_Look 33_compressed.jpg caused a File Not Found Error
3132 of 15543
./compressed_images/Saint Laurent_Spring 2016 Ready-to-Wear_Look 35_compressed.jpg caused a File Not Found Error
./compressed_images/Saint Laurent_Spring 2016 Ready-to-Wear_Look 36_compressed.jpg caused a File Not Found Error
3135 of 15543
./compressed_images/Saint Laurent_Spring 2016 Ready-to-Wear_Look 38_compressed.jpg caused a File Not Found Error
./compressed_images/Saint Laurent_Spring 2016 Ready-to-Wear_Look 39_compressed.jpg caused a File Not Found Erro

...

13996 of 15543
13997 of 15543
13998 of 15543
13999 of 15543
14000 of 15543
14001 of 15543
14002 of 15543
14003 of 15543
14004 of 15543
14005 of 15543
14006 of 15543
14007 of 15543
14008 of 15543
14009 of 15543
14010 of 15543
14011 of 15543
14012 of 15543
14013 of 15543
14014 of 15543
14015 of 15543
14016 of 15543
14017 of 15543
14018 of 15543
14019 of 15543
14020 of 15543
14021 of 15543
14022 of 15543
14023 of 15543
14024 of 15543
14025 of 15543
./compressed_images/Chanel_Pre-Fall 2015_Look 21_compressed.jpg caused a Runtime Error
14027 of 15543
14028 of 15543
14029 of 15543
14030 of 15543
14031 of 15543
14032 of 15543
14033 of 15543
14034 of 15543
14035 of 15543
14036 of 15543
14037 of 15543
14038 of 15543
14039 of 15543
14040 of 15543
14041 of 15543
14042 of 15543
14043 of 15543
14044 of 15543
14045 of 15543
14046 of 15543
14047 of 15543
14048 of 15543
14049 of 15543
14050 of 15543
./compressed_images/Chanel_Pre-Fall 2015_Look 46_compressed.jpg caused a Runtime Error
14052 of 15543
1

14406 of 15543
14407 of 15543
14408 of 15543
14409 of 15543
14410 of 15543
14411 of 15543
14412 of 15543
14413 of 15543
14414 of 15543
14415 of 15543
14416 of 15543
14417 of 15543
14418 of 15543
14419 of 15543
14420 of 15543
14421 of 15543
14422 of 15543
14423 of 15543
14424 of 15543
14425 of 15543
14426 of 15543
14427 of 15543
14428 of 15543
14429 of 15543
14430 of 15543
14431 of 15543
14432 of 15543
14433 of 15543
14434 of 15543
14435 of 15543
14436 of 15543
14437 of 15543
14438 of 15543
14439 of 15543
14440 of 15543
14441 of 15543
14442 of 15543
14443 of 15543
14444 of 15543
14445 of 15543
14446 of 15543
14447 of 15543
14448 of 15543
14449 of 15543
14450 of 15543
14451 of 15543
14452 of 15543
14453 of 15543
14454 of 15543
14455 of 15543
14456 of 15543
14457 of 15543
14458 of 15543
14459 of 15543
14460 of 15543
14461 of 15543
14462 of 15543
14463 of 15543
14464 of 15543
14465 of 15543
14466 of 15543
14467 of 15543
14468 of 15543
14469 of 15543
14470 of 15543
14471 of 15543
14472 of 1

14856 of 15543
14857 of 15543
14858 of 15543
14859 of 15543
14860 of 15543
14861 of 15543
14862 of 15543
14863 of 15543
14864 of 15543
14865 of 15543
14866 of 15543
14867 of 15543
14868 of 15543
14869 of 15543
14870 of 15543
14871 of 15543
14872 of 15543
14873 of 15543
14874 of 15543
14875 of 15543
14876 of 15543
14877 of 15543
14878 of 15543
14879 of 15543
14880 of 15543
14881 of 15543
14882 of 15543
14883 of 15543
14884 of 15543
14885 of 15543
14886 of 15543
14887 of 15543
14888 of 15543
14889 of 15543
14890 of 15543
14891 of 15543
14892 of 15543
14893 of 15543
14894 of 15543
14895 of 15543
14896 of 15543
14897 of 15543
14898 of 15543
14899 of 15543
14900 of 15543
14901 of 15543
14902 of 15543
14903 of 15543
14904 of 15543
14905 of 15543
14906 of 15543
14907 of 15543
14908 of 15543
14909 of 15543
14910 of 15543
14911 of 15543
14912 of 15543
14913 of 15543
14914 of 15543
14915 of 15543
14916 of 15543
14917 of 15543
14918 of 15543
14919 of 15543
./compressed_images/Gucci_Fall 2018 Mens

15016 of 15543
15017 of 15543
15018 of 15543
15019 of 15543
15020 of 15543
15021 of 15543
15022 of 15543
15023 of 15543
15024 of 15543
15025 of 15543
15026 of 15543
15027 of 15543
15028 of 15543
15029 of 15543
15030 of 15543
15031 of 15543
15032 of 15543
./compressed_images/Gucci_Resort 2018_Look 24_compressed.jpg caused a File Not Found Error
15034 of 15543
15035 of 15543
15036 of 15543
15037 of 15543
15038 of 15543
15039 of 15543
15040 of 15543
15041 of 15543
15042 of 15543
15043 of 15543
15044 of 15543
15045 of 15543
15046 of 15543
15047 of 15543
./compressed_images/Gucci_Resort 2018_Look 39_compressed.jpg caused a File Not Found Error
15049 of 15543
15050 of 15543
15051 of 15543
15052 of 15543
15053 of 15543
15054 of 15543
15055 of 15543
15056 of 15543
15057 of 15543
15058 of 15543
15059 of 15543
15060 of 15543
15061 of 15543
15062 of 15543
15063 of 15543
15064 of 15543
15065 of 15543
15066 of 15543
15067 of 15543
15068 of 15543
15069 of 15543
15070 of 15543
15071 of 15543
15072 of

15464 of 15543
15465 of 15543
15466 of 15543
15467 of 15543
15468 of 15543
15469 of 15543
15470 of 15543
15471 of 15543
15472 of 15543
15473 of 15543
15474 of 15543
15475 of 15543
15476 of 15543
15477 of 15543
15478 of 15543
15479 of 15543
15480 of 15543
15481 of 15543
15482 of 15543
15483 of 15543
15484 of 15543
15485 of 15543
15486 of 15543
15487 of 15543
15488 of 15543
15489 of 15543
15490 of 15543
15491 of 15543
15492 of 15543
15493 of 15543
15494 of 15543
15495 of 15543
15496 of 15543
15497 of 15543
15498 of 15543
15499 of 15543
15500 of 15543
15501 of 15543
15502 of 15543
15503 of 15543
15504 of 15543
15505 of 15543
15506 of 15543
15507 of 15543
15508 of 15543
15509 of 15543
15510 of 15543
15511 of 15543
15512 of 15543
15513 of 15543
15514 of 15543
15515 of 15543
15516 of 15543
15517 of 15543
15518 of 15543
15519 of 15543
15520 of 15543
15521 of 15543
15522 of 15543
15523 of 15543
15524 of 15543
15525 of 15543
15526 of 15543
15527 of 15543
15528 of 15543
15529 of 15543
15530 of 1

In [6]:
new_path

'./tensor_flow_detected_models/Gucci_Fall 2015 Menswear_Look 36_detected.jpg'

In [7]:
tensor_flow_detected_model_directory.shape

(13348, 1)

In [8]:
len(runtime_errors)

898

## running this again, to get the first 1297

In [9]:
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("./resnet50_coco_best_v2.0.1.h5")
detector.loadModel()

start = time.time()
for image_index in range(1297):
#for image_index in range(1):#for running once
    filename = image_directory.file_path[image_index]
    designer = image_directory.designer[image_index]
    season= image_directory.season[image_index]
    look = image_directory.look[image_index]
    
    #compress the image and save it to a new folder
    image = cv2.imread(filename,cv2.IMREAD_COLOR)
    compressed_image=cv2.resize(image,None,fx=0.1,fy=0.1)
    compressed_image_filepath='./compressed_images/{}_{}_{}_compressed.jpg'.format(designer,season,look)
    cv2.imwrite(compressed_image_filepath, compressed_image)
    compressed_directory=compressed_directory.append(pd.DataFrame(data=[compressed_image_filepath],columns=['compressed_image_filepath'],index=[image_index]))
    #0.09733414649963379 for 1 iteration to get to here
    #~25 minutes 21 seconds for 15,000 images 
    
    
    #pick out the models using tensorflow
    try:
        detections = detector.detectObjectsFromImage(input_image=compressed_image_filepath,
                                                output_image_path="./close_up/new.jpg",
                                                extract_detected_objects=True)
    except (RuntimeError,SystemError,ValueError): 
        print(compressed_image_filepath+' caused a Runtime Error')
        runtime_errors.append(compressed_image_filepath)
        continue
        
    model_image=find_the_model(detections)
    new_path = "./tensor_flow_detected_models/{}_{}_{}_detected.jpg".format(designer,season,look)
    try: 
        os.rename(model_image,new_path)
    except FileNotFoundError:
        print(compressed_image_filepath+' caused a File Not Found Error')
        runtime_errors.append(compressed_image_filepath)
        continue
    tensor_flow_detected_model_directory=tensor_flow_detected_model_directory.append(pd.DataFrame(data=[new_path],columns=['detected_image_filepath'],index=[image_index]))
    #7.613460063934326 for 1 iteration to get to here
    #32.868998976018695 hours for 15,542 images 
    #12.986150979995728 seconds for uncompressed images,taking 56.06409959197045 hours
    print(str(image_index)+' of '+ str(image_directory.shape[0]))
end = time.time()
compressed_directory.to_csv('./compressed_images/directory.csv')
tensor_flow_detected_model_directory.to_csv('./tensor_flow_detected_models/directory.csv')
print(end - start)
 

0 of 15543
1 of 15543
2 of 15543
3 of 15543
4 of 15543
5 of 15543
6 of 15543
7 of 15543
8 of 15543
9 of 15543
10 of 15543
11 of 15543
12 of 15543
13 of 15543
14 of 15543
15 of 15543
16 of 15543
17 of 15543
./compressed_images/Céline_Spring 2018 Ready-to-Wear_Look 19_compressed.jpg caused a Runtime Error
19 of 15543
20 of 15543
21 of 15543
22 of 15543
23 of 15543
24 of 15543
25 of 15543
26 of 15543
27 of 15543
28 of 15543
29 of 15543
30 of 15543
31 of 15543
32 of 15543
33 of 15543
34 of 15543
35 of 15543
36 of 15543
37 of 15543
38 of 15543
39 of 15543
40 of 15543
41 of 15543
42 of 15543
43 of 15543
44 of 15543
45 of 15543
46 of 15543
47 of 15543
48 of 15543
49 of 15543
50 of 15543
51 of 15543
52 of 15543
53 of 15543
54 of 15543
55 of 15543
56 of 15543
57 of 15543
58 of 15543
59 of 15543
60 of 15543
61 of 15543
62 of 15543
63 of 15543
64 of 15543
65 of 15543
66 of 15543
67 of 15543
68 of 15543
69 of 15543
70 of 15543
71 of 15543
72 of 15543
73 of 15543
74 of 15543
75 of 15543
76 of 15543

433 of 15543
434 of 15543
435 of 15543
436 of 15543
437 of 15543
./compressed_images/Calvin Klein Collection_Spring 2015 Ready-to-Wear_Look 8_compressed.jpg caused a Runtime Error
./compressed_images/Calvin Klein Collection_Spring 2015 Ready-to-Wear_Look 9_compressed.jpg caused a Runtime Error
440 of 15543
441 of 15543
442 of 15543
443 of 15543
444 of 15543
445 of 15543
446 of 15543
447 of 15543
./compressed_images/Calvin Klein Collection_Spring 2015 Ready-to-Wear_Look 18_compressed.jpg caused a Runtime Error
449 of 15543
450 of 15543
451 of 15543
452 of 15543
453 of 15543
454 of 15543
455 of 15543
456 of 15543
./compressed_images/Calvin Klein Collection_Spring 2015 Ready-to-Wear_Look 27_compressed.jpg caused a Runtime Error
458 of 15543
./compressed_images/Calvin Klein Collection_Spring 2015 Ready-to-Wear_Look 29_compressed.jpg caused a Runtime Error
./compressed_images/Calvin Klein Collection_Spring 2015 Ready-to-Wear_Look 30_compressed.jpg caused a Runtime Error
./compressed_images/

817 of 15543
818 of 15543
819 of 15543
820 of 15543
./compressed_images/CALVIN KLEIN 205W39NYC_Fall 2017 Ready-to-Wear_Look 24_compressed.jpg caused a Runtime Error
822 of 15543
823 of 15543
824 of 15543
825 of 15543
826 of 15543
827 of 15543
828 of 15543
829 of 15543
./compressed_images/CALVIN KLEIN 205W39NYC_Fall 2017 Ready-to-Wear_Look 33_compressed.jpg caused a Runtime Error
831 of 15543
./compressed_images/CALVIN KLEIN 205W39NYC_Fall 2017 Ready-to-Wear_Look 35_compressed.jpg caused a Runtime Error
./compressed_images/CALVIN KLEIN 205W39NYC_Fall 2017 Ready-to-Wear_Look 36_compressed.jpg caused a Runtime Error
834 of 15543
835 of 15543
836 of 15543
837 of 15543
838 of 15543
839 of 15543
840 of 15543
841 of 15543
842 of 15543
843 of 15543
844 of 15543
845 of 15543
846 of 15543
847 of 15543
848 of 15543
./compressed_images/CALVIN KLEIN 205W39NYC_Fall 2017 Ready-to-Wear_Look 52_compressed.jpg caused a Runtime Error
850 of 15543
851 of 15543
852 of 15543
853 of 15543
854 of 15543
855 of

1193 of 15543
1194 of 15543
1195 of 15543
1196 of 15543
1197 of 15543
1198 of 15543
1199 of 15543
1200 of 15543
1201 of 15543
1202 of 15543
1203 of 15543
1204 of 15543
1205 of 15543
1206 of 15543
1207 of 15543
1208 of 15543
1209 of 15543
1210 of 15543
1211 of 15543
1212 of 15543
1213 of 15543
1214 of 15543
1215 of 15543
1216 of 15543
1217 of 15543
1218 of 15543
1219 of 15543
1220 of 15543
1221 of 15543
1222 of 15543
1223 of 15543
1224 of 15543
1225 of 15543
1226 of 15543
1227 of 15543
1228 of 15543
1229 of 15543
1230 of 15543
1231 of 15543
1232 of 15543
1233 of 15543
1234 of 15543
1235 of 15543
1236 of 15543
./compressed_images/Versace_Spring 2015 Ready-to-Wear_Look 41_compressed.jpg caused a Runtime Error
1238 of 15543
1239 of 15543
1240 of 15543
1241 of 15543
1242 of 15543
1243 of 15543
1244 of 15543
1245 of 15543
1246 of 15543
1247 of 15543
1248 of 15543
1249 of 15543
1250 of 15543
1251 of 15543
./compressed_images/Versace_Fall 2015 Ready-to-Wear_Look 7_compressed.jpg caused a Runti

In [14]:
len(runtime_errors)

1002

In [13]:
#Anything that caused an error, I included here. I should have labeled at which point the images were lost,
#But since I did not, I included the lost images in both libraries for easy access. Every image in the list 
# will not exist in the tensor_flow_detected_models file path. However, file not found errors only occured before
#the tensor flow step. Therefore, there are about 330 images that threw a file not found error that exist in the 
#compressed image library that do not exist in the tensor_flow_detected_models library. There 1002 images not processed
#All need to be processed again anyway, but I saved them in both directories for easy access.
images_lost_in_processing=pd.DataFrame(data=runtime_errors,columns=['data lost due to errors'])
images_lost_in_processing.to_csv('./tensor_flow_detected_models/images_lost_in_processing.csv')
images_lost_in_processing.to_csv('./compressed_images/images_lost_in_processing.csv')

,data lost due to errors
0,./compressed_images/Versace_Fall 2015 Ready-to...
1,./compressed_images/Versace_Spring 2015 Menswe...
2,./compressed_images/Versace_Spring 2015 Menswe...
3,./compressed_images/Versace_Fall 2018 Menswear...
4,./compressed_images/Versace_Fall 2017 Ready-to...
5,./compressed_images/Versace_Fall 2017 Ready-to...
6,./compressed_images/Versace_Fall 2017 Ready-to...
7,./compressed_images/Versace_Spring 2017 Ready-...
8,./compressed_images/Versace_Spring 2017 Ready-...
9,./compressed_images/Versace_Spring 2017 Menswe...
